# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

## Preparación

#### Imports y variables globales

In [ ]:
import pandas
import numpy
import sklearn
import networkx as nx
from sklearn import preprocessing, model_selection, naive_bayes

semilla = 86

#### Lectura y procesamiento inicial de los datos brutos

In [ ]:
#Leemos los archivos
aristas = pandas.read_csv('data/political-books-nodes.csv')
vertices = pandas.read_csv('data/political-books-edges.csv')

#Borramos la columna ID
del(aristas['Id'])

#Mostramos las primeras 35 filas
aristas.head(35)

#### Selección y validación de los datos brutos

In [ ]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(aristas) != len(set(aristas['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = aristas['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivos = aristas['political_ideology']

## Inicio del entrenamiento
#### Codificación del objetivo

In [ ]:

"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivos = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivos_codificados = codificador_objetivos.fit_transform(objetivos)

#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos, de entrenamiento y de prueba

In [ ]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivos_entrenamiento,
 objetivos_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivos_codificados,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=.33,
        stratify=objetivos_codificados
)

## Naive Bayes
#### No relacional

In [ ]:
suavizado = 0.3

clasif_NB = naive_bayes.MultinomialNB(alpha=suavizado)
clasif_NB.fit(atributos_entrenamiento, objetivos_entrenamiento)

#Calculamos la cantidad de ejemplos para cada clase y los logaritmos
for clase, cantidad_ejemplos_clase, log_probabilidad_clase in zip(
        classif_NB.classes_, classif_NB.class_count_, classif_NB.class_log_prior_):
    print("Cantidad de ejemplos para la clase", clase, ":", cantidad_ejemplos_clase)
    print("Logaritmo de la probabilidad aprendida para la clase", clase, ":", log_probabilidad_clase)
